In [2]:
import matplotlib
%matplotlib inline
np = pd.np
np.norm = np.linalg.norm
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split


# Pandas as a Database
Rather than installing postrgres and Django and configuring all that
You can just use a Pandas collection of DataFrame tables as your database

In [40]:
import pandas as pd
import re
regex = re.compile(r'[\b_^][Ii][Dd][\b_$]')
pacs = pd.DataFrame.from_csv('public.raw_committees.csv')  # no ID that I can find
pac_id_cols = [col for col in pacs.columns if regex.search(col)]
print(pac_id_cols)
pacs_scraped = pd.DataFrame.from_csv('public.raw_committees_scraped.csv')  # id
pac_scraped_id_cols = [col for col in pacs_scraped.columns if regex.search(col)]
print(pac_scraped_id_cols)
candidates = pd.DataFrame.from_csv('public.raw_candidate_filings.csv')  # id_nmbr
candidate_id_cols = [col for col in candidates.columns if regex.search(col)]
print(candidate_id_cols)
trans = pd.DataFrame.from_csv('public.raw_committee_transactions_ammended_transactions.csv')
trans_id_cols = [col for col in trans.columns if regex.search(col)]
print(trans_id_cols)


[]
[]
[]
[]


# Original_ID?

So it looks like there are multiple revisions for many of the "unique" original_id. So to consolidate those revisions into unqiue records with unique IDs (take the most recent revision as the official record):

In [12]:
trans = pd.DataFrame.from_csv('public.raw_committee_transactions_ammended_transactions.csv')
trans.describe()

,original_id,amount,aggregate_amount,contributor_payee_committee_id,filer_id,intrst_rate,check_nbr,addr_book_agent_name,zip,zip_plus_four,exp_date
count,2997.000000,2997.000000,2997.000000,160.000000,2997.000000,9,5.080000e+02,0,2355.000000,150.000000,0
mean,1885216.702369,1530.411655,4623.306210,7128.418750,9703.807808,0,5.985057e+06,NaN,90213.352017,5940.713333,NaN
std,211274.747411,10621.464988,30598.010565,6719.423466,6866.094987,0,1.331107e+08,NaN,21051.288729,2989.624421,NaN
min,71433.000000,-597.630000,0.000000,3.000000,10.000000,0,0.000000e+00,NaN,2117.000000,1.000000,NaN
25%,1864267.000000,35.000000,118.000000,629.000000,2480.000000,0,1.012750e+03,NaN,97038.000000,3498.000000,NaN
50%,1929490.000000,150.000000,250.000000,4749.000000,11508.000000,0,1.167500e+03,NaN,97219.000000,6628.000000,NaN
75%,1979015.000000,500.000000,977.650000,14931.000000,17001.000000,0,1.979250e+03,NaN,97401.000000,8652.000000,NaN
max,2049455.000000,452550.000000,543057.000000,17370.000000,17454.000000,0,3.000184e+09,NaN,99362.000000,9998.000000,NaN


In [14]:
filtered_trans = []
for id in trans.original_id.unique():
    rows = sorted(trans[trans.original_id == id].iterrows(), key=lambda x:x[1].attest_date, reverse=True)
    filtered_trans += [rows[0][1]]
filtered_trans = pd.DataFrame(filtered_trans)
print(len(trans) / float(len(filtered_trans)))
print(filtered_trans.describe())

1.11
          original_id         amount  aggregate_amount  \
count     2700.000000    2700.000000       2700.000000   
mean   1884926.093333    1398.428496       4339.031659   
std     222078.767780   10643.772144      31247.914447   
min      71433.000000    -597.630000          0.000000   
25%    1864223.750000      35.000000        108.290000   
50%    1942541.000000     125.000000        238.805000   
75%    1979260.500000     500.000000        767.900000   
max    2049455.000000  452550.000000     543057.000000   

       contributor_payee_committee_id      filer_id  intrst_rate  \
count                      129.000000   2700.000000            9   
mean                      7872.085271   9612.684444            0   
std                       6991.062308   6897.686139            0   
min                          3.000000     10.000000            0   
25%                        470.000000   2189.000000            0   
50%                       4831.000000  11487.000000            0

In [30]:
df = filtered_trans
filer_sums = df.groupby('filer_id').amount.sum()
print(pacs.columns)
print(df.columns)
print([(pacs[pacs.committee_id == id].committee_name, amount) for (id, amount) in sorted(filer_sums.iteritems(), key=lambda x:x[1], reverse=True)])


Index([u'committee_name', u'committee_type', u'committee_subtype',
       u'candidate_office', u'candidate_office_group', u'filing_date',
       u'organization_filing Date', u'treasurer_first_name',
       u'treasurer_last_name', u'treasurer_mailing_address',
       u'treasurer_work_phone', u'treasurer_fax', u'candidate_first_name',
       u'candidate_last_name', u'candidate_maling_address',
       u'candidate_work_phone', u'candidate_residence_phone', u'candidate_fax',
       u'candidate_email', u'active_election', u'measure'],
      dtype='object')
Index([u'original_id', u'tran_date', u'tran_status', u'filer',
       u'contributor_payee', u'sub_type', u'amount', u'aggregate_amount',
       u'contributor_payee_committee_id', u'filer_id', u'attest_by_name',
       u'attest_date', u'review_by_name', u'review_date', u'due_date',
       u'occptn_ltr_date', u'pymt_sched_txt', u'purp_desc', u'intrst_rate',
       u'check_nbr', u'tran_stsfd_ind', u'filed_by_name', u'filed_date',
       u'add

AttributeError: 'DataFrame' object has no attribute 'committee_id'